In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(7*7*32, 10)

    def forward(self, x):
        out = self.maxpool1(self.relu1(self.conv1(x)))
        out = self.maxpool2(self.relu2(self.conv2(out)))
        out = out.view(-1, 7*7*32)
        out = self.fc(out)
        return out

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 10

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]) 

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Model, loss function, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# Training loop
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, epochs, i+1, len(train_loader), loss.item()))

Epoch [1/10], Step [100/938], Loss: 0.3565
Epoch [1/10], Step [200/938], Loss: 0.0600
Epoch [1/10], Step [300/938], Loss: 0.0726
Epoch [1/10], Step [400/938], Loss: 0.1187
Epoch [1/10], Step [500/938], Loss: 0.0698
Epoch [1/10], Step [600/938], Loss: 0.0369
Epoch [1/10], Step [700/938], Loss: 0.0258
Epoch [1/10], Step [800/938], Loss: 0.1524
Epoch [1/10], Step [900/938], Loss: 0.0386
Epoch [2/10], Step [100/938], Loss: 0.0715
Epoch [2/10], Step [200/938], Loss: 0.0690
Epoch [2/10], Step [300/938], Loss: 0.0926
Epoch [2/10], Step [400/938], Loss: 0.0169
Epoch [2/10], Step [500/938], Loss: 0.2173
Epoch [2/10], Step [600/938], Loss: 0.0240
Epoch [2/10], Step [700/938], Loss: 0.1348
Epoch [2/10], Step [800/938], Loss: 0.1478
Epoch [2/10], Step [900/938], Loss: 0.1018
Epoch [3/10], Step [100/938], Loss: 0.0864
Epoch [3/10], Step [200/938], Loss: 0.0611
Epoch [3/10], Step [300/938], Loss: 0.0054
Epoch [3/10], Step [400/938], Loss: 0.0155
Epoch [3/10], Step [500/938], Loss: 0.0299
Epoch [3/10

In [6]:
# Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 98.93 %
